# Engenharia de Features

Este notebook tem como objetivo transformar os dados consolidados em variáveis úteis para o modelo de regressão.

**Principais Transformações:**
* **Cálculo de Cubagem:** Determinação do peso cúbico e do peso final.
* **Cálculo de Distância:** Aplicação da fórmula de Haversine para obter a distância entre vendedor e cliente.
* **Seleção Final:** Filtragem das colunas, mantendo apenas as *features* essenciais e a variável *target*.

In [1]:
import pandas as pd
from pathlib import Path
import sys
import os

project_root = os.path.abspath("../../..")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.T3.regressao.features import calcular_peso_cubado, haversine_vectorized, selecionar_peso_final

In [2]:
BASE_PATH = Path("../../../")
DATA_INTERIM = BASE_PATH / "data" / "interim" / "T3" / "regressao"
DATA_PROCESSED = BASE_PATH / "data" / "processed" / "T3" / "regressao"

In [3]:
dataset_merged = pd.read_csv(DATA_INTERIM / "dataset_merged.csv")

## 3.1. Peso Final

No transporte de cargas, o frete é cobrado com base no maior valor entre o **peso físico** real e o **peso cubado**.

**Fórmula utilizada:**
$$Peso\ Cubado\ (kg) = \frac{Comprimento \times Largura \times Altura}{Fator\ de\ Cubagem}$$

**Critérios adotados:**
* **Fator de Cubagem (6000):** É o padrão oficial da IATA e utilizado pela maioria das transportadoras rodoviárias e Correios no Brasil para converter volume ($cm^3$) em peso taxável ($kg$).
* **Conversão de Unidade:** A fórmula original resulta em quilogramas. Como a coluna de peso original (`product_weight_g`) está em gramas, multiplicamos o resultado do peso cúbico por **1000** para manter a consistência das unidades.
* **Peso Final:** Definido como o valor máximo entre o peso real e o peso cúbico calculado (`max(peso_real, peso_cubado)`).

In [4]:
df = dataset_merged.copy()

df['peso_cubado_g'] = calcular_peso_cubado(
    df['product_length_cm'], 
    df['product_height_cm'], 
    df['product_width_cm'], 
    fator_cubagem=6000
)

df['final_weight_g'] = selecionar_peso_final(df['product_weight_g'], df['peso_cubado_g'])

## 3.2. Distância

Para estimar a distância de entrega, utilizamos a **Fórmula de Haversine**.

**Fórmula utilizada:**
$$d = 2r \arcsin\left(\sqrt{\sin^2\left(\frac{\Delta\phi}{2}\right) + \cos(\phi_1)\cos(\phi_2)\sin^2\left(\frac{\Delta\lambda}{2}\right)}\right)$$

Onde:
* $r$: Raio da Terra (~6371 km)
* $\phi$: Latitude (em radianos)
* $\lambda$: Longitude (em radianos)

In [5]:
df['distance_km'] = haversine_vectorized(
    df['lat_seller'].values, 
    df['lng_seller'].values, 
    df['lat_customer'].values, 
    df['lng_customer'].values,
    R=6371.0 # Raio da terra em KM
)

## Seleção de Features e Target

In [6]:
cols_finais = [
    'price',
    'final_weight_g',
    'customer_state',
    'seller_state',
    'distance_km',
    'freight_value'
]

df_final = df[cols_finais].copy()
df_final.head()

,price,final_weight_g,customer_state,seller_state,distance_km,freight_value
0,58.90,650.0,RJ,SP,301.504681,13.29
1,239.90,30000.0,SP,SP,585.563937,19.93
2,199.00,3050.0,MG,MG,312.343511,17.87
3,12.99,400.0,SP,SP,293.168420,12.79
4,199.90,7000.0,SP,PR,646.163463,18.14


## Exportação dos Dados

In [7]:
df_final.to_csv(DATA_PROCESSED / "freight_prediction_abt.csv", index=False)